# Obteniendo el **número de dosis aplicada** contra COVID-19 por semana epidemiológica de todo el Perú

## 0. Cargar librerías y funciones

In [1]:
import pandas as pd
import numpy as np

import functions as fn

In [3]:
def vac_dose(vac_url):
    """
    Función que toma la dirección del dataset de vacunados y devuelve el TOTAL DE DOSIS APLICADAS 
    (1ra,2nda y 3era dosis) por semana epidemiológica
    """
    vac_cols = ['fecha_vacunacion', 'dosis']                     
    df_vac = fn.read_largeCSV_file(vac_url, ',', vac_cols)    
    lst_vac = fn.df_into_chunks(df_vac)               
 
    for idx, chunk in enumerate(lst_vac):
        chunk = fn.variable_fecha_ymd(chunk, 'fecha_vacunacion')

    for idx, chunk in enumerate(lst_vac):
        chunk = fn.date_to_epiweek(chunk, "fecha_vacunacion")
        
    return lst_vac 
    

## 1. Cargar el dataset de **personas vacunadas (1era y 2nda dosis)** por semana epidemiológica

In [4]:
vac_url = "RawData/TB_VACUNACION_COVID19.csv"
vacDose = vac_dose(vac_url)
del vac_url

/home/xxotto/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/home/xxotto/Documents/GitHub/covid19-peru/functions.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['epi_year','epi_week']] = df[date_name].apply(lambda date_name : Week.fromdate(date_name).weektuple()).tolist()
/home/xxotto/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value

Dataset resultante:

In [5]:
print(vacDose[0].head())

   dosis  epi_year  epi_week
0      2      2021        29
1      2      2021        24
2      2      2021        23
3      2      2021        30
4      1      2021        18


## 2. Crosstab del número de dosis (1,2 y 3) respecto a la semana epidemiológica

In [6]:
def vacDose_chunks(dfs_vac):
    """
    Devuelve un dataframe con el TOTAL DE VACUNADOS por DEPARTAMENTO 
    del PERÚ (recibe una lista de dataframes o chunks)
    """
    var_holder = {}     # Diccionario para guardar nombres                                             
    lst_epi_vac = []    # Lista de dfs para cada sumatoria de chunks
                                         
    for i, chunk in enumerate(dfs_vac):
        var_holder['epi_vac_' + str(i)]= pd.crosstab(index = [chunk['epi_year'], chunk['epi_week']],
                                                     columns = [chunk['dosis']])
        lst_epi_vac.append(var_holder['epi_vac_' + str(i)])

    merged_epivac = pd.concat(lst_epi_vac, axis=1)  # Merge all dfs
    merged_epivac = merged_epivac.fillna(0).astype(np.int64)
    merged_epivac = merged_epivac.groupby(level=0, axis=1).sum()

    return merged_epivac

In [7]:
vacDosexsemEpi = vacDose_chunks(vacDose)
print(vacDosexsemEpi.head(50))

dosis                    1        2       3
epi_year epi_week                          
2021     6          105755        1       0
         7           92287        4       0
         8          123882       19       0
         9           35142    93044       0
         10          74236    80891       0
         11         133491   120498       0
         12          80275    38699       0
         13          25407    65417       0
         14          69399   101913       0
         15          56525   111395       0
         16         117949    29749       0
         17         278608    60094       0
         18         283547    51057       0
         19         394910   106779       0
         20         452380   174047       0
         21         563328   239365       0
         22         316680   209601       0
         23         515857   560825       0
         24         253304   618950       0
         25         344743   364838       0
         26         544901   447

Guardar el dataframe:

In [8]:
vacDosexsemEpi.to_csv('Data/dosis_vacuna_x_semanaEpi.csv')